In [105]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn import ensemble, metrics, linear_model
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from kneed import KneeLocator
from matplotlib.ticker import PercentFormatter
import pingouin as pg
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
import spacy
sp = spacy.load('en_core_web_sm')

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline

import time
from tqdm import tqdm
from pandas.api.types import is_numeric_dtype
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, average_precision_score, auc, f1_score
import shap

# not advisable, but we do it for a cleaner report
# warnings regarding non-convergence of some classifiers used are muted
import warnings 
warnings.filterwarnings("ignore")

In [107]:
# read 2020 procurement data
df_jan_mar_2020 = pd.read_csv(
    'Bid Notice and Award Details Jan-Mar 2020.csv', encoding = "ISO-8859-1")
df_apr_jun_2020 = pd.read_csv(
    'Bid Notice and Award Details Apr-Jun 2020.csv', encoding = "ISO-8859-1")
df_jul_sept_2020 = pd.read_csv(
    'Bid Notice and Award Details Jul-Sep 2020.csv', encoding = "ISO-8859-1")
df_oct_dec_2020 = pd.read_csv(
    'Bid Notice and Award Details Oct-Dec 2020.csv', encoding = "ISO-8859-1")

In [108]:
# combine 2020 data into 1 dataframe
df_proc_2020 = pd.concat([df_jan_mar_2020, df_apr_jun_2020,
                         df_jul_sept_2020, df_oct_dec_2020])

In [109]:
# change reference ID into str
df_proc_2020['Reference ID'] = df_proc_2020['Reference ID'].astype(str)

In [110]:
# make column names of df_proc_2020 consistent
df_jul_sept_2017 = pd.read_csv(
    'Bid Notice and Award Details Jul-Sep 2017.csv', encoding = "ISO-8859-1")
col_names_2017 = df_jul_sept_2017.columns
df_proc_2020.columns = col_names_2017

In [111]:
# combine 2020 data into 1 dataframe
df_proc_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810863 entries, 0 to 236718
Data columns (total 40 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Organization Name                810863 non-null  object 
 1   Reference ID                     810863 non-null  object 
 2   Solicitation No.                 810797 non-null  object 
 3   Notice Title                     810863 non-null  object 
 4   Publish Date                     810863 non-null  object 
 5   Classification                   810863 non-null  object 
 6   Notice Type                      810863 non-null  object 
 7   Business Category                810863 non-null  object 
 8   Funding Source                   810863 non-null  object 
 9   Funding Instrument               810863 non-null  object 
 10  Procurement Mode                 810863 non-null  object 
 11  Trade Agreement                  810863 non-null  object 
 12  Ap

In [112]:
# Change Contract Amount Column of SUC Procurement Data to Numeric Column
df_proc_2020['Contract Amount'] = df_proc_2020[
    'Contract Amount'].replace(',','', regex=True)
df_proc_2020['Contract Amount'] = df_proc_2020[
    'Contract Amount'].replace('-','', regex=True)
df_proc_2020['Contract Amount'] = df_proc_2020[
    'Contract Amount'].replace(' ','', regex=True)
df_proc_2020['Contract Amount'] = df_proc_2020[
    'Contract Amount'].replace('',np.nan, regex=True)
df_proc_2020['Contract Amount'] = df_proc_2020[
    'Contract Amount'].replace('NULL',np.nan, regex=True)
df_proc_2020['Contract Amount'] = df_proc_2020[
    'Contract Amount'].astype(float)

In [113]:
# calculate for Contract Duration to Days Multiplier
# for consistency: 1 month = 30 days: 1 year = 12 months = 360 days
df_proc_2020['Calendar Type'].unique()
num_days = {'Day/s': 1, 'Month/s': 30, 'Year/s': 360, np.nan: 1}
df_proc_2020['Days Multiplier'] = df_proc_2020['Calendar Type']
df_proc_2020['Days Multiplier'] = df_proc_2020[
    'Days Multiplier'].replace(num_days)
df_proc_2020['Contract Duration (Days)'] = df_proc_2020.apply(
    lambda row: (row['Days Multiplier']*row['Contract Duration']), axis=1)

In [114]:
# convert date columns to date time data types
date_cols = ['Publish Date', 'PreBid Date', 'Closing Date', 'Publish '
             'Date(Award)', 'Award Date', 'Notice to Proceed Date', 
             'Contract Efectivity Date', 'Contract End Date']
for i in date_cols:
    df_proc_2020[i] = pd.to_datetime(df_proc_2020[i].str.replace(
        '24:|25:|26:|27:|28:|29:|30:|31:|32:|33:|34:|35:|36:'
        '|37:|38:|39:|40:|41:|42:|43:|44:|45:|46:|47:|48:|49:|'
        '50:|51:|52:|53:|54:|55:|56:|57:|58:|59:','00:'))

In [115]:
# difference (days) between prebid date and publish date
df_proc_2020['Publish-PreBid (Days)'] = df_proc_2020[
    'PreBid Date'] - df_proc_2020['Publish Date']
df_proc_2020['Publish-PreBid (Days)'] = pd.to_numeric(df_proc_2020[
    'Publish-PreBid (Days)'].dt.days, downcast='integer')

# difference (days) between closing date and prebid date
df_proc_2020['PreBid-Closing (Days)'] = df_proc_2020[
    'Closing Date'] - df_proc_2020['PreBid Date']
df_proc_2020['PreBid-Closing (Days)'] = pd.to_numeric(df_proc_2020[
    'PreBid-Closing (Days)'].dt.days, downcast='integer')

# difference (days) between notice to proceed date and award date
# df_proc_2020['Award-NTP (Days)'] = df_proc_2020[
#     'Notice to Proceed Date'] - df_proc_2020['Award Date']
# df_proc_2020['Award-NTP (Days)'] = pd.to_numeric(df_proc_2020[
#     'Award-NTP (Days)'].dt.days, downcast='integer')

In [116]:
# create Year and Quarter Columns
# df_proc_2020['Year'] = df_proc_2020['Publish Date'].dt.year
df_proc_2020['Quarter'] = df_proc_2020['Publish Date'].dt.quarter
df_proc_2020['Month'] = df_proc_2020['Publish Date'].dt.month

# # change Quarter column into string
# df_proc_2020['Quarter'] = df_proc_2020['Quarter'].map({1: '1st', 2: '2nd',
#                                                    3: '3rd', 4: '4th'})

In [117]:
# drop duplicates
df_proc_2020 = df_proc_2020.drop_duplicates(subset=['Reference ID'], keep='first')

In [118]:
# create deep copy of cleaned dataframe
df_proc_2020_clean = df_proc_2020.copy(deep=True)

In [119]:
# check cleaned dataframe
df_proc_2020_clean

,Organization Name,Reference ID,Solicitation No.,Notice Title,Publish Date,Classification,Notice Type,Business Category,Funding Source,Funding Instrument,...,Contract Efectivity Date,Contract End Date,Reason for Award,Award Status,Days Multiplier,Contract Duration (Days),Publish-PreBid (Days),PreBid-Closing (Days),Quarter,Month
0,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - BENG...,6776642,20PF0030,20PF0030 Convergence and Special Support Progr...,2020-01-01,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),National Expenditure Program (NEP) for the suc...,...,NaT,NaT,NaN,NaN,1,106.0,2.0,11.0,1,1
1,PROVINCE OF BATANES,6779335,2019-1375-12 (02),S & D of Construction Materials for the Comple...,2020-01-01,Goods,Invitation to Bid (ITB),Construction Materials and Supplies,Government of the Philippines (GOP),Budget for the Contract Approved by the Sanggu...,...,NaT,NaT,NaN,NaN,1,30.0,NaN,NaN,1,1
2,PROVINCE OF BATANES,6779368,2019-1357-11 (02),S & D of Materials for the Fabrication of Love...,2020-01-01,Goods,Invitation to Bid (ITB),Construction Materials and Supplies,Government of the Philippines (GOP),Budget for the Contract Approved by the Sanggu...,...,NaT,NaT,NaN,NaN,1,30.0,NaN,NaN,1,1
3,PROVINCE OF BATANES,6779402,2019-1393-12 (02),S & D of Construction Materials for the Constr...,2020-01-01,Goods,Invitation to Bid (ITB),Construction Materials and Supplies,Government of the Philippines (GOP),Budget for the Contract Approved by the Sanggu...,...,NaT,NaT,NaN,NaN,1,45.0,NaN,NaN,1,1
4,DEPARTMENT OF HEALTH - MAIN,6780338,IB NO. 2020-042-A,PROCUREMENT OF SERVICE PROVIDER FOR THE HAULIN...,2020-01-01,Goods - General Support Services,Invitation to Bid (ITB),Services,Government of the Philippines (GOP),National Expenditure Program (NEP) for the suc...,...,NaT,NaT,NaN,NaN,30,360.0,7.0,14.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236714,DEPARTMENT OF SOCIAL WELFARE AND DEVELOPMENT -...,7392868,DSWD 4B 2020-09-0814,Purchase and Delivery of NFA Rice Stockpile in...,2020-12-31,Goods,Invitation to Apply for Eligibility and to Bid,Food Stuff,Government of the Philippines (GOP),General Appropriations Act,...,2020-09-25,2020-10-02,Negotiated Procurement - Agency to Agency (Sec...,Updated,1,7.0,NaN,NaN,4,12
236715,DEPARTMENT OF SOCIAL WELFARE AND DEVELOPMENT -...,7392871,DSWD 4B 2020-11-1148,Purchase and Delivery of 120 Sacks of NFA Ric...,2020-12-31,Goods,Invitation to Apply for Eligibility and to Bid,Food Stuff,Government of the Philippines (GOP),General Appropriations Act,...,2020-12-02,2020-12-09,Negotiated Procurement - Agency to Agency (Sec...,Updated,1,7.0,NaN,NaN,4,12
236716,CIVIL SERVICE COMMISSION - REGION VI,7392901,Postage Stamps (FC 102),Procurement of Postage Stamps (FC 102),2020-12-31,Goods,Invitation to Apply for Eligibility and to Bid,Mail and Cargo Transport Services,Government of the Philippines (GOP),Corporate Budget for the Contract Approved by ...,...,2020-02-07,2020-02-21,Negotiated Procurement - Agency to Agency.,Updated,1,10.0,NaN,NaN,4,12
236717,PHILIPPINE STATISTICS AUTHORITY - LA UNION PRO...,7392924,2020-12-108,"Provision of Meals, Snacks, Venue and Accommod...",2020-12-31,Goods,Invitation to Apply for Eligibility and to Bid,Hotel and Lodging and Meeting Facilities,Government of the Philippines (GOP),General Appropriations Act,...,2020-12-28,2020-12-29,LCRQ,Updated,1,2.0,NaN,NaN,4,12


In [120]:
# check for null values
df_proc_2020_clean.isnull().sum()

Organization Name                       0
Reference ID                            0
Solicitation No.                       66
Notice Title                            0
Publish Date                            0
Classification                          0
Notice Type                             0
Business Category                       0
Funding Source                          0
Funding Instrument                      0
Procurement Mode                        0
Trade Agreement                         0
Approved Budget of the Contract         0
Area of Delivery                    46219
Contract Duration                    3449
Calendar Type                       19540
Line Item No                       365564
Item Name                          365565
Item Desc                          365570
Quantity                           365564
UOM                                365564
Item Budget                        365564
PreBid Date                        436917
Closing Date                      

In [121]:
# replace null values under Area of Delivery with "Others"
df_proc_2020_clean['Area of Delivery'] = df_proc_2020_clean[
    'Area of Delivery'].fillna('Others')

# replace null values under Contract Duration with mean of Contract Duration
# column
df_proc_2020_clean['Contract Duration'] = df_proc_2020_clean[
    'Contract Duration'].fillna(
    value=df_proc_2020_clean['Contract Duration'].mean())

# replace null values under Item Name with "None"
df_proc_2020_clean['Item Name'] = df_proc_2020_clean[
    'Item Name'].fillna('None')

# replace null values under Item Desc with "None"
df_proc_2020_clean['Item Desc'] = df_proc_2020_clean[
    'Item Desc'].fillna('None')

# replace null values under Quantity with mean of Quantity Column
df_proc_2020_clean['Quantity'] = df_proc_2020_clean[
    'Quantity'].fillna(value=df_proc_2020_clean['Quantity'].mean())

# replace null values under Item Budget with mean of Item Budget Column
df_proc_2020_clean['Item Budget'] = df_proc_2020_clean[
    'Item Budget'].fillna(value=df_proc_2020_clean['Item Budget'].mean())

# replace null values under Award Title with "None"
df_proc_2020_clean['Award Title'] = df_proc_2020_clean[
    'Award Title'].fillna('None')

# replace null values under Award Type with "None"
df_proc_2020_clean['Award Type'] = df_proc_2020_clean[
    'Award Type'].fillna('None')

# replace null values under Awardee Corporate Title with "None"
df_proc_2020_clean['Awardee Corporate Title'] = df_proc_2020_clean[
    'Awardee Corporate Title'].fillna('None')

# replace null values under Contract Amount with 0
df_proc_2020_clean['Contract Amount'] = df_proc_2020_clean[
    'Contract Amount'].fillna(0)

# replace null values under Reason for Award with "Not Awarded"
df_proc_2020_clean['Reason for Award'] = df_proc_2020_clean[
    'Reason for Award'].fillna('Not Awarded')

# replace null values under Award Status with "Not Awarded"
df_proc_2020_clean['Award Status'] = df_proc_2020_clean[
    'Award Status'].fillna('Not Awarded')

# replace null values under Contract Duration (Days) with mean of Contract 
# Duration (Days) column
df_proc_2020_clean['Contract Duration (Days)'] = df_proc_2020_clean[
    'Contract Duration (Days)'].fillna(
    value=df_proc_2020_clean['Contract Duration (Days)'].mean())

# replace null values under Publish-PreBid (Days) with 0
df_proc_2020_clean['Publish-PreBid (Days)'] = df_proc_2020_clean[
    'Publish-PreBid (Days)'].fillna(0)

# replace null values under PreBid-Closing (Days) with 0
df_proc_2020_clean['PreBid-Closing (Days)'] = df_proc_2020_clean[
    'PreBid-Closing (Days)'].fillna(0)

In [122]:
# check if null values were successfully replaced
df_proc_2020_clean.isnull().sum()

Organization Name                       0
Reference ID                            0
Solicitation No.                       66
Notice Title                            0
Publish Date                            0
Classification                          0
Notice Type                             0
Business Category                       0
Funding Source                          0
Funding Instrument                      0
Procurement Mode                        0
Trade Agreement                         0
Approved Budget of the Contract         0
Area of Delivery                        0
Contract Duration                       0
Calendar Type                       19540
Line Item No                       365564
Item Name                               0
Item Desc                               0
Quantity                                0
UOM                                365564
Item Budget                             0
PreBid Date                        436917
Closing Date                      

In [123]:
# check info of cleaned dataframe
df_proc_2020_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567012 entries, 0 to 236718
Data columns (total 46 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Organization Name                567012 non-null  object        
 1   Reference ID                     567012 non-null  object        
 2   Solicitation No.                 566946 non-null  object        
 3   Notice Title                     567012 non-null  object        
 4   Publish Date                     567012 non-null  datetime64[ns]
 5   Classification                   567012 non-null  object        
 6   Notice Type                      567012 non-null  object        
 7   Business Category                567012 non-null  object        
 8   Funding Source                   567012 non-null  object        
 9   Funding Instrument               567012 non-null  object        
 10  Procurement Mode                 567012 non-

In [124]:
# write to csv cleaned file
# df_proc_2020_clean.to_csv('Cleaned PhilGEPS 2020 Data.csv')

In [126]:
# filter here
# filter for Manila only
df_dept = df_proc_2020_clean.loc[df_proc_2020_clean['Area of Delivery'].str.contains('Metro Manila')]
df_dept

,Organization Name,Reference ID,Solicitation No.,Notice Title,Publish Date,Classification,Notice Type,Business Category,Funding Source,Funding Instrument,...,Contract Efectivity Date,Contract End Date,Reason for Award,Award Status,Days Multiplier,Contract Duration (Days),Publish-PreBid (Days),PreBid-Closing (Days),Quarter,Month
53,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777812,HB 1912-061,Reconditioning of Bow Thruster for BRP Hydrogr...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Maritime Spare Parts,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,30,30.0,0.0,0.0,1,1
54,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777816,MB 1911-032,GNSS re-observation of GCPs in Region 1 and Is...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Surveying Services,Government of the Philippines (GOP),General Appropriations Act,...,2020-02-21,2020-05-21,Lowest Calculated Responsive Bid,Updated,30,90.0,0.0,0.0,1,1
55,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777819,MB 1911-030,Upgrade of the Philippine Active Geodetic Netw...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Surveying Services,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,30,120.0,0.0,0.0,1,1
56,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777862,MB 1911-031,Upgrade of the Philippine Active Geodetic Netw...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Surveying Instruments,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,30,120.0,0.0,0.0,1,1
121,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6779306,MB 1911-029,Re-levelling of Cotabato to Kabacan to Datu Pa...,2020-01-02,Goods,Invitation to Bid (ITB),Surveying Services,Government of the Philippines (GOP),General Appropriations Act,...,2020-03-02,2020-07-02,Lowest Calculated Responsive Bid,Updated,30,120.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236615,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392721,21-187,Contract ID No. 21OB0187 - Construction/Rehab...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,90.0,0.0,0.0,4,12
236618,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392725,21-188,Contract ID No. 21OB0188 - Construction/Rehab...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,120.0,0.0,0.0,4,12
236621,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392728,21-189,Contract ID No. 21OB0189 - Construction/Rehab...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,90.0,0.0,0.0,4,12
236623,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392730,21-199,Contract ID No. 21OB0199 - Rehabilitation of ...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,90.0,0.0,0.0,4,12


In [127]:
# define target column
df_dept_target = df_dept['Notice Status'].map({'Closed': 0, 'Cancelled': 0, 'Awarded': 1, 'Failed': 0, 'Shortlisted': 0, 'Active': 0})
df_dept_target.value_counts()

0    51198
1    11373
Name: Notice Status, dtype: int64

In [128]:
# drop target column
df_dept.drop('Notice Status', axis=1, inplace=True)
df_dept

,Organization Name,Reference ID,Solicitation No.,Notice Title,Publish Date,Classification,Notice Type,Business Category,Funding Source,Funding Instrument,...,Contract Efectivity Date,Contract End Date,Reason for Award,Award Status,Days Multiplier,Contract Duration (Days),Publish-PreBid (Days),PreBid-Closing (Days),Quarter,Month
53,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777812,HB 1912-061,Reconditioning of Bow Thruster for BRP Hydrogr...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Maritime Spare Parts,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,30,30.0,0.0,0.0,1,1
54,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777816,MB 1911-032,GNSS re-observation of GCPs in Region 1 and Is...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Surveying Services,Government of the Philippines (GOP),General Appropriations Act,...,2020-02-21,2020-05-21,Lowest Calculated Responsive Bid,Updated,30,90.0,0.0,0.0,1,1
55,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777819,MB 1911-030,Upgrade of the Philippine Active Geodetic Netw...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Surveying Services,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,30,120.0,0.0,0.0,1,1
56,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777862,MB 1911-031,Upgrade of the Philippine Active Geodetic Netw...,2020-01-02,Goods - General Support Services,Invitation to Bid (ITB),Surveying Instruments,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,30,120.0,0.0,0.0,1,1
121,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6779306,MB 1911-029,Re-levelling of Cotabato to Kabacan to Datu Pa...,2020-01-02,Goods,Invitation to Bid (ITB),Surveying Services,Government of the Philippines (GOP),General Appropriations Act,...,2020-03-02,2020-07-02,Lowest Calculated Responsive Bid,Updated,30,120.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236615,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392721,21-187,Contract ID No. 21OB0187 - Construction/Rehab...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,90.0,0.0,0.0,4,12
236618,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392725,21-188,Contract ID No. 21OB0188 - Construction/Rehab...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,120.0,0.0,0.0,4,12
236621,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392728,21-189,Contract ID No. 21OB0189 - Construction/Rehab...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,90.0,0.0,0.0,4,12
236623,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392730,21-199,Contract ID No. 21OB0199 - Rehabilitation of ...,2020-12-31,Civil Works,Invitation to Bid (ITB),Construction Projects,Government of the Philippines (GOP),General Appropriations Act,...,NaT,NaT,Not Awarded,Not Awarded,1,90.0,0.0,0.0,4,12


In [130]:
# extract numeric columns from data from cleaned data
numeric = ['Approved Budget of the Contract', 'Contract Duration (Days)',
           'Publish-PreBid (Days)', 'PreBid-Closing (Days)', 'Quarter', 'Month']
df_proc_2020_numeric = df_dept[numeric]

In [132]:
# extract categoric columns from data from cleaned data
categoric = ['Classification', 'Notice Type', 'Business Category',
               'Funding Source', 'Funding Instrument',
               'Procurement Mode', 'Trade Agreement']
df_proc_2020_categoric = df_dept[categoric]

In [133]:
# check categoric dataframe
df_proc_2020_categoric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62571 entries, 53 to 236645
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Classification      62571 non-null  object
 1   Notice Type         62571 non-null  object
 2   Business Category   62571 non-null  object
 3   Funding Source      62571 non-null  object
 4   Funding Instrument  62571 non-null  object
 5   Procurement Mode    62571 non-null  object
 6   Trade Agreement     62571 non-null  object
dtypes: object(7)
memory usage: 3.8+ MB


In [134]:
# create dummy variables for categoric datafrmae
df_proc_2020_categoric_ohe = pd.get_dummies(
    df_proc_2020_categoric, columns = categoric)

In [135]:
# check one-hot encoded categoric dataframe
df_proc_2020_categoric_ohe

,Classification_Civil Works,Classification_Consulting Services,Classification_Goods,Classification_Goods - General Support Services,Notice Type_Invitation to Apply for Eligibility and to Bid,Notice Type_Invitation to Bid (ITB),Notice Type_Notice for Negotiated Procurement,Notice Type_Request for Expression of Interest,Notice Type_Request for Proposal (RFP),Notice Type_Request for Quotation (RFQ),...,Procurement Mode_Shopping,Procurement Mode_Shopping - Ordinary/Regular Office Supplies & Equipment (Sec. 52.1.b),Procurement Mode_Shopping - Unforeseen Contingency (Sec. 52.1.a),Trade Agreement_ADB Guidelines on the use of Consultants,Trade Agreement_ADB Procurement Guidelines,Trade Agreement_Implementing Rules and Regulations,Trade Agreement_International / Executive Agreement,Trade Agreement_Others,Trade Agreement_WB Guidelines on the use of Consultants,Trade Agreement_WB Procurement Guidelines
53,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
54,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
55,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
56,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
121,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236615,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
236618,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
236621,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
236623,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [136]:
# concat dataframe
df_proc_2020_feature = pd.concat([df_dept['Organization Name'],
                          df_dept['Reference ID'],
                          df_proc_2020_numeric, df_proc_2020_categoric_ohe], axis=1)

In [137]:
# reset index (for training and splitting, and backtracking)
df_proc_2020_feature_indexed = df_proc_2020_feature.reset_index(drop=True)
df_proc_2020_feature_indexed

,Organization Name,Reference ID,Approved Budget of the Contract,Contract Duration (Days),Publish-PreBid (Days),PreBid-Closing (Days),Quarter,Month,Classification_Civil Works,Classification_Consulting Services,...,Procurement Mode_Shopping,Procurement Mode_Shopping - Ordinary/Regular Office Supplies & Equipment (Sec. 52.1.b),Procurement Mode_Shopping - Unforeseen Contingency (Sec. 52.1.a),Trade Agreement_ADB Guidelines on the use of Consultants,Trade Agreement_ADB Procurement Guidelines,Trade Agreement_Implementing Rules and Regulations,Trade Agreement_International / Executive Agreement,Trade Agreement_Others,Trade Agreement_WB Guidelines on the use of Consultants,Trade Agreement_WB Procurement Guidelines
0,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777812,1800000.00,30.0,0.0,0.0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777816,2736000.00,90.0,0.0,0.0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777819,9000000.00,120.0,0.0,0.0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6777862,9000000.00,120.0,0.0,0.0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,NATIONAL MAPPING AND RESOURCE INFORMATION AUTH...,6779306,5400000.00,120.0,0.0,0.0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62566,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392721,4949747.44,90.0,0.0,0.0,4,12,1,0,...,0,0,0,0,0,1,0,0,0,0
62567,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392725,4949743.95,120.0,0.0,0.0,4,12,1,0,...,0,0,0,0,0,1,0,0,0,0
62568,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392728,4941871.32,90.0,0.0,0.0,4,12,1,0,...,0,0,0,0,0,1,0,0,0,0
62569,DEPARTMENT OF PUBLIC WORKS AND HIGHWAYS - METR...,7392730,4949495.37,90.0,0.0,0.0,4,12,1,0,...,0,0,0,0,0,1,0,0,0,0


In [138]:
# extract feature column
df_feature = df_proc_2020_feature_indexed.drop(['Organization Name', 'Reference ID'], axis=1)
df_feature

,Approved Budget of the Contract,Contract Duration (Days),Publish-PreBid (Days),PreBid-Closing (Days),Quarter,Month,Classification_Civil Works,Classification_Consulting Services,Classification_Goods,Classification_Goods - General Support Services,...,Procurement Mode_Shopping,Procurement Mode_Shopping - Ordinary/Regular Office Supplies & Equipment (Sec. 52.1.b),Procurement Mode_Shopping - Unforeseen Contingency (Sec. 52.1.a),Trade Agreement_ADB Guidelines on the use of Consultants,Trade Agreement_ADB Procurement Guidelines,Trade Agreement_Implementing Rules and Regulations,Trade Agreement_International / Executive Agreement,Trade Agreement_Others,Trade Agreement_WB Guidelines on the use of Consultants,Trade Agreement_WB Procurement Guidelines
0,1800000.00,30.0,0.0,0.0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,2736000.00,90.0,0.0,0.0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,9000000.00,120.0,0.0,0.0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,9000000.00,120.0,0.0,0.0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5400000.00,120.0,0.0,0.0,1,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62566,4949747.44,90.0,0.0,0.0,4,12,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
62567,4949743.95,120.0,0.0,0.0,4,12,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
62568,4941871.32,90.0,0.0,0.0,4,12,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
62569,4949495.37,90.0,0.0,0.0,4,12,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [140]:
# defined target column
df_dept_target

53        0
54        1
55        0
56        0
121       1
         ..
236615    0
236618    0
236621    0
236623    0
236645    0
Name: Notice Status, Length: 62571, dtype: int64